In [ ]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import *

# Cell to create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 ASSIGNMENT 1 DUSTIN")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/08/22 18:48:26 WARN Utils: Your hostname, DESKTOP-3ADPNV0 resolves to a loopback address: 127.0.1.1; using 172.25.21.27 instead (on interface eth0)
22/08/22 18:48:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


In [ ]:
import pandas as pd
import statsmodels.api as sm

In [ ]:
yellow = spark.read.parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_credit')
yellow_sample = spark.read.parquet('../../mast30034-project-1-dustintano10/data/curated/yellow/yellow_sample')

In [ ]:
yellow_sample = yellow_sample.withColumn('pickup_hour', hour("tpep_pickup_datetime"))
yellow_sample = yellow_sample.withColumn('dropoff_hour', hour("tpep_dropoff_datetime"))
yellow_sample = yellow_sample.withColumn('is_weekend', F.when(yellow.is_weekend == 'false', 0).otherwise(1))

yellow_sample = yellow_sample.withColumn('Win', F.when(yellow.Win == 'No', 0).otherwise(1))

In [ ]:
yellow_sample_pandas = yellow_sample.toPandas()


In [16]:
yellow_sample_pandas.rename(columns = {'margin_victory/loss':'score_margin'}, inplace = True )
yellow_sample_pandas

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,total_amount,is_weekend,trip_length,Date,Start(ET),Attendance,Win,score_margin,pickup_hour,dropoff_hour
0,2018-10-26 00:00:39,2018-10-26 00:20:58,3.67,65,225,16.5,4.45,22.25,False,20.3167,2018-10-26,19:30,19812,No,-28,0,0
1,2018-10-26 00:30:26,2018-10-26 00:37:25,1.10,161,234,6.5,1.00,8.80,False,6.9833,2018-10-26,19:30,19812,No,-28,0,0
2,2018-10-26 00:15:27,2018-10-26 00:26:08,2.90,148,87,11.5,3.20,16.00,False,10.6833,2018-10-26,19:30,19812,No,-28,0,0
3,2018-10-26 00:34:47,2018-10-26 00:51:25,1.80,232,79,12.0,1.00,14.30,False,16.6333,2018-10-26,19:30,19812,No,-28,0,0
4,2018-10-26 00:22:33,2018-10-26 00:47:43,5.79,68,116,21.0,4.46,26.76,False,25.1667,2018-10-26,19:30,19812,No,-28,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641336,2019-03-24 23:36:46,2019-03-24 23:47:53,7.20,132,95,21.0,4.45,26.75,True,11.1167,2019-03-24,12:00,18263,No,-11,23,23
641337,2019-03-24 23:54:40,2019-03-24 23:58:35,1.02,144,113,5.5,1.86,11.16,True,3.9167,2019-03-24,12:00,18263,No,-11,23,23
641338,2019-03-24 09:14:27,2019-03-24 09:36:37,3.61,113,237,16.0,3.04,23.34,True,22.1667,2019-03-24,12:00,18263,No,-11,9,9
641339,2019-03-24 10:03:10,2019-03-24 10:06:48,0.74,234,186,4.5,1.24,9.54,True,3.6333,2019-03-24,12:00,18263,No,-11,10,10


In [28]:
x_columns = ["trip_distance" ,"PULocationID", "DOLocationID", "fare_amount", "total_amount", 
             "is_weekend", "trip_length", "Attendance", "Win", "pickup_hour", "dropoff_hour", "score_margin"]
y = yellow_sample_pandas["tip_amount"]

def get_stats():
    x = yellow_sample_pandas[x_columns]
    results = sm.OLS(y, x).fit()
    print(results.summary())
    
get_stats()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [5]:
yellow.groupBy('Date').count().orderBy('Date')


Date,count
2018-10-17,182129
2018-10-20,169804
2018-10-26,181843
2018-10-29,156363
2018-10-31,164490
2018-11-05,161472
2018-11-11,148113
2018-11-20,166765
2018-11-23,99489
2018-12-01,180716


In [6]:
# converts the pickup_hours into categorical data as there are correlation that could be seen from analysis

# drop the tpep pickup and dropoff columns

yellow = yellow.withColumn('pickup_hour', hour("tpep_pickup_datetime"))
yellow = yellow.withColumn('dropoff_hour', hour("tpep_dropoff_datetime"))

yellow = yellow.drop('tpep_pickup_datetime', 'tpep_dropoff_datetime', 'Start(ET)', 'margin_victory/loss', 'Attendance')

In [7]:
# convert boolean values into 0 and 1

yellow = yellow.withColumn('is_weekend', F.when(yellow.is_weekend == 'false', 0).otherwise(1))

yellow = yellow.withColumn('Win', F.when(yellow.Win == 'No', 0).otherwise(1))

yellow

trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,total_amount,is_weekend,trip_length,Date,Win,pickup_hour,dropoff_hour
2.56,162,263,16.5,4.58,22.88,0,25.3833,2018-12-25,0,0,1
0.9,163,162,5.0,1.45,7.25,0,4.6,2018-12-25,0,9,9
0.63,100,48,4.0,0.96,5.76,0,2.6167,2018-12-25,0,10,10
8.08,161,13,23.5,0.0,24.3,0,15.8167,2018-12-25,0,10,11
2.13,162,142,11.5,1.0,13.3,0,14.6667,2018-12-25,0,15,16
2.88,230,236,11.0,2.36,14.16,0,10.8167,2018-12-25,0,0,0
0.98,161,229,6.0,1.36,8.16,0,6.1667,2018-12-25,0,0,0
1.64,246,107,9.5,0.0,10.3,0,12.4167,2018-12-25,0,3,4
0.83,237,141,5.0,2.0,7.8,0,3.9833,2018-12-25,0,17,17
1.1,237,236,5.5,1.58,7.88,0,3.8167,2018-12-25,0,17,17


In [8]:
# use all 2018 data as a train set

# then use the first game of 2019 as the test set

yellow_train = yellow.where((F.col('Date') < '2019-01-11'))

yellow_test = yellow.where((F.col('Date') == '2019-01-11'))


In [9]:
yellow_train = yellow_train.drop('Date')

In [10]:
yellow_test = yellow_test.drop('Date')

In [11]:
yellow_train

trip_distance,PULocationID,DOLocationID,fare_amount,tip_amount,total_amount,is_weekend,trip_length,Win,pickup_hour,dropoff_hour
2.56,162,263,16.5,4.58,22.88,0,25.3833,0,0,1
0.9,163,162,5.0,1.45,7.25,0,4.6,0,9,9
0.63,100,48,4.0,0.96,5.76,0,2.6167,0,10,10
8.08,161,13,23.5,0.0,24.3,0,15.8167,0,10,11
2.13,162,142,11.5,1.0,13.3,0,14.6667,0,15,16
2.88,230,236,11.0,2.36,14.16,0,10.8167,0,0,0
0.98,161,229,6.0,1.36,8.16,0,6.1667,0,0,0
1.64,246,107,9.5,0.0,10.3,0,12.4167,0,3,4
0.83,237,141,5.0,2.0,7.8,0,3.9833,0,17,17
1.1,237,236,5.5,1.58,7.88,0,3.8167,0,17,17


In [12]:
from pyspark.ml.feature import VectorAssembler

features = 'features'

input_cols = ['fare_amount', 'trip_distance', 'total_amount', 'is_weekend', 'trip_length', 'Win',
             'PULocationID', 'DOLocationID', 'pickup_hour', 'dropoff_hour']

vectorAssembler = VectorAssembler(
    inputCols = input_cols, 
    outputCol = features)

v_yellow_train = vectorAssembler.transform(yellow_train.dropna('any'))

v_yellow_train = v_yellow_train.select(['features', 'tip_amount'])

v_yellow_train.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[16.5,2.56,22.88,...|      4.58|
|[5.0,0.9,7.25,0.0...|      1.45|
|[4.0,0.63,5.76,0....|      0.96|
+--------------------+----------+
only showing top 3 rows



In [21]:
vectorAssembler = VectorAssembler(
    inputCols = input_cols, 
    outputCol = features)

v_yellow_test = vectorAssembler.transform(yellow_test.dropna('any'))

v_yellow_test = v_yellow_test.select(['features', 'tip_amount'])

v_yellow_test.show(3)

+--------------------+----------+
|            features|tip_amount|
+--------------------+----------+
|[13.0,2.88,18.5,0...|       3.7|
|[10.0,2.73,13.56,...|      2.26|
|[8.0,2.03,11.16,0...|      1.86|
+--------------------+----------+
only showing top 3 rows



In [13]:
from pyspark.ml.regression import LinearRegression
lm = LinearRegression(featuresCol='features', labelCol='tip_amount').fit(v_yellow_train)


22/08/22 13:43:34 WARN Instrumentation: [9ed71451] regParam is zero, which might cause numerical instability and overfitting.
22/08/22 13:43:35 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/08/22 13:43:35 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


22/08/22 13:43:40 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [14]:
# Access coefficients
pd.DataFrame(
    data=[lm.intercept] + list(lm.coefficients),
    index=['intercept'] + input_cols,
    columns=['coefficient']
)

,coefficient
intercept,-0.356892
fare_amount,-0.775146
trip_distance,0.000229
total_amount,0.735339
is_weekend,0.112685
trip_length,0.049369
Win,-0.002104
PULocationID,0.000110
DOLocationID,0.000071
pickup_hour,-0.016068


In [20]:
trainingSummary = lm.summary

print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
print("MSE: %f" % trainingSummary.meanSquaredError)
print("MAE: %f" % trainingSummary.meanAbsoluteError)



RMSE: 0.517739
r2: 0.863578
MSE: 0.268054
MAE: 0.305733


In [18]:
v_yellow_train.describe().show()

+-------+------------------+
|summary|        tip_amount|
+-------+------------------+
|  count|           2493024|
|   mean|2.1022256384216456|
| stddev|1.4017429974266153|
|    min|               0.0|
|    max|             411.0|
+-------+------------------+



In [24]:
from pyspark.ml.evaluation import RegressionEvaluator
lm_predictions = lm.transform(v_yellow_test)
lm_predictions.select("prediction","tip_amount","features").show(5)
lm_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="tip_amount",metricName="r2")
print("R Squared (R2) on test data = %g" % lm_evaluator.evaluate(lm_predictions))

+------------------+----------+--------------------+
|        prediction|tip_amount|            features|
+------------------+----------+--------------------+
| 3.911862283017643|       3.7|[13.0,2.88,18.5,0...|
|2.3485717572441147|      2.26|[10.0,2.73,13.56,...|
| 2.077656941933478|      1.86|[8.0,2.03,11.16,0...|
|2.5557676585083837|      2.49|[7.0,1.39,10.79,0...|
|3.6198110799468903|      4.06|[19.0,5.88,24.36,...|
+------------------+----------+--------------------+
only showing top 5 rows



R Squared (R2) on test data = 0.873226


In [43]:
test_result = lm.evaluate(v_yellow_test)

print("R Squared (R2) on test data = %g" % test_result.r2)
print("Root Mean Squared Error (RMSE) on test data = %g" % test_result.rootMeanSquaredError)
print("Mean Squared Error (MSE) on test data = %g" % test_result.meanSquaredError)
print("Mean Absolute Error (MAE) on test data = %g" % test_result.meanAbsoluteError)

R Squared (R2) on test data = 0.873226
Root Mean Squared Error (RMSE) on test data = 0.519531
Mean Squared Error (MSE) on test data = 0.269913
Mean Absolute Error (MAE) on test data = 0.306498


In [31]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'tip_amount').fit(v_yellow_train)

dt_predictions = dt.transform(v_yellow_test)

dt_predictions.select("prediction","tip_amount","features").show(5)

+------------------+----------+--------------------+
|        prediction|tip_amount|            features|
+------------------+----------+--------------------+
|3.3008385782169656|       3.7|[13.0,2.88,18.5,0...|
| 2.333849971327231|      2.26|[10.0,2.73,13.56,...|
| 1.900132687040225|      1.86|[8.0,2.03,11.16,0...|
| 1.900132687040225|      2.49|[7.0,1.39,10.79,0...|
|3.5703602680886886|      4.06|[19.0,5.88,24.36,...|
+------------------+----------+--------------------+
only showing top 5 rows



In [46]:
dt_evaluator_r2 = RegressionEvaluator(
    labelCol="tip_amount", predictionCol="prediction", metricName="r2")

dt_evaluator_rmse = RegressionEvaluator(
    labelCol="tip_amount", predictionCol="prediction", metricName="rmse")

dt_evaluator_mse = RegressionEvaluator(
    labelCol="tip_amount", predictionCol="prediction", metricName="mse")

dt_evaluator_mae = RegressionEvaluator(
    labelCol="tip_amount", predictionCol="prediction", metricName="mae")


prediction_results_dt_r2 = dt_evaluator_r2.evaluate(dt_predictions)

prediction_results_dt_rmse = dt_evaluator_rmse.evaluate(dt_predictions)

prediction_results_dt_mse = dt_evaluator_mse.evaluate(dt_predictions)

prediction_results_dt_mae = dt_evaluator_mae.evaluate(dt_predictions)


print("R Squared (R2) on test data = %g" % prediction_results_dt_r2)

print("Root Mean Squared Error (RMSE) on test data = %g" % prediction_results_dt_rmse)

print("Mean Squared Error (MSE) on test data = %g" % prediction_results_dt_mse)

print("Mean Absolute Error (MAE) on test data = %g" % prediction_results_dt_mae)




R Squared (R2) on test data = 0.616291
Root Mean Squared Error (RMSE) on test data = 0.903852
Mean Squared Error (MSE) on test data = 0.816948
Mean Absolute Error (MAE) on test data = 0.396183


In [48]:
# feature with index number 2 is most important
# it is total_amount

print(input_cols)
dt.featureImportances

['fare_amount', 'trip_distance', 'total_amount', 'is_weekend', 'trip_length', 'Win', 'PULocationID', 'DOLocationID', 'pickup_hour', 'dropoff_hour']


SparseVector(10, {0: 0.1799, 1: 0.0041, 2: 0.8155, 4: 0.0005})